# Example of upper limit computation.

In [ ]:
from __future__ import annotations

import matplotlib.pyplot as plt
import numpy as np

import zfit
from zfit.loss import ExtendedUnbinnedNLL
from zfit.minimize import Minuit

from utils import plotfitresult, plotlimit, pltdist
import wrappers as wr

import os
os.chdir("/home/cms-jovyan/hepstats/hepstats/src")

# from hepstats.hypotests import UpperLimit
from hepstats.hypotests.calculators import AsymptoticCalculator
from hepstats.hypotests.core.functions import upperlimit
from hepstats.utils import POI, POIarray
# from hepstats.hypotests.parameters import POI, POIarray

os.chdir("/home/cms-jovyan/hepstats/hepstats/notebooks/hypotests")
print(os.listdir())

In [ ]:
plt.rcParams["figure.figsize"] = (9, 8)
plt.rcParams["font.size"] = 16

### Fit of a Gaussian signal over an exponential background:

In [ ]:
bounds = (0.1, 3.0)

# Data and signal

np.random.seed(0)
tau = -2.0
beta = -1 / tau
data = np.random.exponential(beta, 300)
peak = np.random.normal(1.2, 0.1, 10)
data = np.concatenate((data, peak))
data = data[(data > bounds[0]) & (data < bounds[1])]

In [ ]:
pltdist(data, bins=80, bounds=bounds)

In [ ]:
obs = zfit.Space("x", limits=bounds)

In [ ]:
zfit_lambda = zfit.Parameter("lambda", -2.0, -4.0, -1.0)
zfit_Nsig = zfit.Parameter("Nsig", 1.0, -20.0, len(data))
zfit_Nbkg = zfit.Parameter("Nbkg", len(data), 0.0, len(data) * 1.1)

signal = zfit.pdf.Gauss(obs=obs, mu=1.2, sigma=0.1).create_extended(zfit_Nsig)
background = zfit.pdf.Exponential(obs=obs, lambda_=zfit_lambda).create_extended(zfit_Nbkg)
tot_model = zfit.pdf.SumPDF([signal, background])

In [ ]:
Nsig = wr.Parameter.from_zfit(zfit_Nsig)
Nbkg = wr.Parameter.from_zfit(zfit_Nbkg)
lambda_ = wr.Parameter.from_zfit(zfit_lambda)
    
bkg = wr.Parameter.concat([Nbkg, lambda_], floating=True)

params = {"signal" : Nsig, "background" : bkg}

In [ ]:
unwrap_params = lambda p: {"Nsig" : p["signal"], "Nbkg" : p["background"][0], "lambda" : p["background"][1]}

data_dict = {"M" : data}
models_dict = {"M" : wr.ExtendedUnbinnedModel.from_zfit(tot_model, unwrap_params)}
data_nll = {"M" : wr.ExtendedUnbinnedNLL(models_dict["M"])}
constraint_nll = lambda params: 0

In [ ]:
# instantation of the calculator
calculator = AsymptoticCalculator(
    data_nll, constraint_nll, params, data_dict,
    models=models_dict, blind=False, beta=10000, unbinned_correction=True
)

bestfit = calculator.bestfit

In [ ]:
min_values = unwrap_params(bestfit.params)

for zfit_param in tot_model.get_params():
    zfit_param.set_value(min_values[zfit_param.name])

nbins = 80
pltdist(data, nbins, bounds)
plotfitresult(tot_model, bounds, nbins)

plt.xlabel("m [GeV/c$^2$]")
plt.ylabel("number of events")
plt.legend()

### Upper limit:

Since the signal yield Nsig = $4.5 \pm 5.8$  is consistent with zero signal we compute the CLs upper limit at 95% confidence level on Nsig.

In [ ]:
poi_range = np.linspace(0, 25, 20)
target_idx = 1
print("poinull_value = {}".format(poi_range[target_idx]))

poinull = POI("signal", poi_range[target_idx])
poialt = POI("signal", 0)

In [ ]:
import time

start = time.time()
prefit_values, postfit_values = calculator.asimov_diagnostics(poinull)
end = time.time()

print("Time elapsed:", end - start)

In [ ]:
print(prefit_values)
print(postfit_values)

In [ ]:
limits, all_pvalues = upperlimit(calculator, "signal", (0.0, 25.0), ntests=20)

In [ ]:
limits

In [ ]:
{'observed': 15.676801586444853,
 'expected': 11.176252816787928,
 'expected_p1': 16.434231523930507,
 'expected_m1': 7.694913571569008,
 'expected_p2': 23.417250569821093,
 'expected_m2': 5.532096420470908}

In [ ]:
all_pvalues

In [ ]:
# ul.upperlimit(alpha=0.05, CLs=True);

In [ ]:
f = plt.figure(figsize=(9, 8))
plotlimit(poi_range, all_pvalues, alpha=0.05, CLs=True)
plt.xlabel("Nsig")
f.savefig("asy_ul.png")

In [ ]:
# f = plt.figure(figsize=(9, 8))
# plotlimit(ul, alpha=0.05, CLs=False)